# Generate Random NETLF Network

In [4]:
import numpy as np
import pandas as pd
import random as rn
import string
import matplotlib.pyplot as plt


In [3]:
N_groups = 10


## Locations and Populations

In [18]:
X = rn.sample(range(0, 1000), N_groups) 
Y = rn.sample(range(0, 1000), N_groups)
names = list(string.ascii_lowercase)[:N_groups]
pops = rn.sample(range(100, 1000), N_groups)

In [19]:
data_nam = {"Group":names}
data_loc = {"Group":names, "X":X,"Y":Y}
data_pop = {"Group":names, "Population":pops}

In [20]:
frame_nam = pd.DataFrame(data_nam) 
frame_loc = pd.DataFrame(data_loc) 
frame_pop = pd.DataFrame(data_pop) 

In [21]:
frame_nam.to_csv("group_names.csv",index=False)
frame_loc.to_csv("group_locations.csv",index=False)
frame_pop.to_csv("group_populations.csv",index=False)

In [6]:
np.set_printoptions(suppress=True)
dist_direct = np.zeros((N_groups,N_groups))
dist_car = np.zeros((N_groups,N_groups))

In [8]:
for i in np.arange(N_groups):
    for j in np.arange(N_groups):
        dist_direct[i,j] = round(np.linalg.norm(np.array((X[i],Y[i]))-np.array((X[j],Y[j]))))
        dist_car[i,j] = round(np.abs(X[i]-X[j])+np.abs(Y[i]-Y[j]))
            

In [9]:
distance_cars = pd.DataFrame(dist_car, columns=names, index=names)
distance_direct = pd.DataFrame(dist_direct, columns=names, index=names)

In [10]:
distance_cars.to_csv("road_dist.csv")
distance_direct.to_csv("euc_dist.csv")

## Births and Deaths

For simplicity I assume that the population tree declines linearly with age. Therefore the proportion of the population in each age bracket can be modelled by a simple line- $y=mx+b$, where $x$ is the proportional size of each age bracket and $y$ is the bracket number (assuming $n$ brackets). As $x$ is a proportion we know:

$\sum_{i=1}^n x_i = 100$

We can then substitute $x$ and get: 

$\frac{\sum_{i=1}^ni - nb}{m} = 100$.

We can get:

$m= \frac{\sum_{i=1}^ni-n^2}{100-n x_n}$,

and 

$b = \frac{x_n\sum_{i=1}^ni -100n}{nx_n-100}$.

Then if we know the number of brackets and a single value for a bracket we can solve for all brackets. For example $n=16$ and $x_{16} =1$. Then $y=-1.423x+17.43$.

In [2]:
N_age_brackets = 16 #number of age brackets
W_brackets = 5 #width of age brackets (years)

In [11]:
xn = 1 #assuming 1% in 75+ pop
b = (xn*np.sum(np.arange(1,N_age_brackets+1))-100*N_age_brackets)/(xn*N_age_brackets-100)
m = (np.sum(np.arange(1,N_age_brackets+1))-N_age_brackets**2)/(100-N_age_brackets*xn)

In [15]:
prop = (np.arange(1,N_age_brackets+1)-b)/m/100

In [27]:
μd = np.zeros(16)

for i in np.arange(16):
    
    if i == 15:
        μd[i] = 1.7/(W_brackets*365)
    else:
        μd[i] = (0.7/(365*W_brackets*prop[i]*100))


In [34]:
μb = np.zeros(16)
α = (12.21/(5*365))/41.7
μb[4:10]= α

In [35]:
α

0.00016044150980585394

In [36]:
birth = {"Daily Rate":μb}
birth_frame = pd.DataFrame(birth)

death = {"Daily Rate":μd}
death_frame = pd.DataFrame(death)

age = {"Age Dist":prop}
age_frame = pd.DataFrame(age)


birth_frame.to_csv("birth_rates.csv",index=False)
death_frame.to_csv("mortality_rates.csv",index=False)
age_frame.to_csv("pop_age_dist.csv",index=False)